In [1]:
#!usr/bin/env python

from __future__ import print_function

In [2]:
from pyrosetta import *
from rosetta import *

# tools
import sys, os
import commands
import random
import numpy as np

# rosetta corrected imports
import rosetta.protocols.membrane
from rosetta.core.pose import Pose
from rosetta.core.scoring import ScoreFunction
from rosetta.core.pack.task import TaskFactory
from rosetta.utility import vector1_bool
from rosetta.core.chemical import aa_from_one_or_three
from rosetta.protocols.minimization_packing import PackRotamersMover
from rosetta.core.pose import PDBInfo
from rosetta.core.chemical import VariantType
from pyrosetta.toolbox import mutate_residue
from pyrosetta.rosetta.core import conformation
from pyrosetta.rosetta.core import chemical

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Import of 'rosetta' as a top-level module is deprecated and may be removed in 2018, import via 'pyrosetta.rosetta'.
  


In [3]:
init( extra_options="-mp:setup:spanfiles TCM_pR.span")

Found rosetta database at: /usr/local/lib/python2.7/dist-packages/pyrosetta-2018.12+release.5ecebca-py2.7-linux-x86_64.egg/pyrosetta/database; using it....
PyRosetta-4 2017 [Rosetta PyRosetta4.Release.python27.ubuntu 2018.12+release.5ecebca5ecebcadacdf48ede3c1981444601bf1cd47ce0d 2018-03-23T13:02:49] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.

core.init: Checking for fconfig files in pwd and ./rosetta/flags

core.init: Rosetta version: PyRosetta4.Release.python27.ubuntu r174 2018.12+release.5ecebca 5ecebcadacdf48ede3c1981444601bf1cd47ce0d http://www.pyrosetta.org 2018-03-23T13:02:49
core.init: command: PyRosetta -ex1 -ex2aro -mp:setup:spanfiles TCM_pR.span -database /usr/local/lib/python2.7/dist-packages/pyrosetta-2018.12+release.5ecebca-py2.7-linux-x86_64.egg/pyrosetta/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=-721343248 seed_offset=0 real_seed=-721343248
c

In [4]:
#Initializing PyMOLMover
from pyrosetta.toolbox import mutate_residue
from pyrosetta import PyMOLMover
my_ip = os.environ["my_ip"]
#my_ip = '127.0.0.1'
pymover = PyMOLMover(my_ip, 65000)

In [5]:
#Load Pose and Add Membrane
pose = pose_from_file("TCM_pR_renum.pdb")
add_memb = rosetta.protocols.membrane.AddMembraneMover()
add_memb.apply( pose )
#Orient Membrane According to Spanfile
init_mem_pos = rosetta.protocols.membrane.MembranePositionFromTopologyMover()
init_mem_pos.apply( pose )
pymover.apply(pose)

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 606 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.717744 seconds.
core.import_pose.import_pose: File 'TCM_pR_renum.pdb' automatically determined to be of type PDB
core.chemical: [ WARNING ] Patch N_Methylation implies it can apply to residue type RYS, but actually applying it fails.
core.chemical: [ WARNING ]    You may want to check your patch definitions.
core.chemical: [ WARNING ] Patch N_Methylation implies it can apply to residue type DRYS, but actually applying it fails.
core.chemical: [ WARNING ]    You may want to check your patch definitions.
core.chemical: [ WARNING ] Patch N_Methylation implies it can apply to residue type DRYS, but actually applying it fails.
core.chemical: [ WARNING ]    You may want to check your patch definitions.
core.chemical: [ WARNING ] Patch N_Methylation implies it can apply to residue type DRYS, but actually applying it f

protocols.membrane.geometry.EmbeddingDef: [ WARNING ] membrane, then your TMspan does not span the membrane!
protocols.membrane.geometry.EmbeddingDef: Computing membrane embedding from TMspan 39 to 56
protocols.membrane.geometry.EmbeddingDef: [ WARNING ] If your starting PDB is already translated into the
protocols.membrane.geometry.EmbeddingDef: [ WARNING ] membrane, then your TMspan does not span the membrane!
protocols.membrane.geometry.EmbeddingDef: Computing membrane embedding from TMspan 77 to 99
protocols.membrane.geometry.EmbeddingDef: Computing membrane embedding from TMspan 104 to 125
protocols.membrane.geometry.EmbeddingDef: Computing membrane embedding from TMspan 134 to 151
protocols.membrane.geometry.EmbeddingDef: Computing membrane embedding from TMspan 163 to 191
protocols.membrane.geometry.EmbeddingDef: Computing membrane embedding from TMspan 204 to 229
protocols.membrane.geometry.Embedding: Span Embedding:
protocols.membrane.geometry.EmbeddingDef: Embedding: center: 

In [6]:
#Load Scorefunction and Score WT
sfxn = create_score_function( "mpframework_no_envsmooth.wts" )
sfxn(pose)
#When testing the "mpframework_smooth_fa_2012", I have been and still do get the error 
#"[ ERROR ] UtilityExitException
#ERROR: ERROR: Failed to find amino acid UNK in EnvSmooth::representative_atom_name"
#Is there another membrane scoring function I could use or alterations I could make to the current one?
#Yay! made a new wts file without the mp_envsmooth term and it works! (placed it in usr/..../pyrosetta/database/scoring/weights/)

core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/PairEPotential/pdb_pair_stats_fine
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
core.scoring.etable: Using alternate parameters: LK_DGFREE in MembEtable construction.
core.scoring.etable: Using alternate parameters: MEMB_LK_DGFREE in MembEtable construction.
core.

-790.8114242397914

In [7]:
#Since this scorefunction works, and the membrane is added, I can likely find a ddg between
#the mutants I made with fixbb, relax, and membrane relax and my wildtype!!!!
#load mutants and show in pymol
S184_mut = pose_from_file("S184_mut_renum.pdb")
add_memb.apply( S184_mut )
init_mem_pos.apply( S184_mut )
pymover.apply(S184_mut)
T188_mut = pose_from_file("T188_mut_renum.pdb")
add_memb.apply( T188_mut )
init_mem_pos.apply( T188_mut)
pymover.apply(T188_mut)
Y191_mut = pose_from_file("Y191_mut_renum.pdb")
add_memb.apply( Y191_mut)
init_mem_pos.apply( Y191_mut)
pymover.apply(Y191_mut)
F195_mut = pose_from_file("F195_mut_renum.pdb")
add_memb.apply( F195_mut)
init_mem_pos.apply( F195_mut)
pymover.apply(F195_mut)
Q183_mut = pose_from_file("Q183_mut_renum.pdb")
add_memb.apply( Q183_mut )
init_mem_pos.apply( Q183_mut )
pymover.apply( Q183_mut )


core.import_pose.import_pose: File 'S184_mut_renum.pdb' automatically determined to be of type PDB
core.chemical: [ WARNING ] Patch N_Methylation implies it can apply to residue type RYS, but actually applying it fails.
core.chemical: [ WARNING ]    You may want to check your patch definitions.
core.chemical: [ WARNING ] Patch N_Methylation implies it can apply to residue type DRYS, but actually applying it fails.
core.chemical: [ WARNING ]    You may want to check your patch definitions.
core.chemical: [ WARNING ] Patch N_Methylation implies it can apply to residue type DRYS, but actually applying it fails.
core.chemical: [ WARNING ]    You may want to check your patch definitions.
core.chemical: [ WARNING ] Patch N_Methylation implies it can apply to residue type DRYS, but actually applying it fails.
core.chemical: [ WARNING ]    You may want to check your patch definitions.
core.chemical: [ WARNING ] Patch N_Methylation implies it can apply to residue type DRYS, but actually applyin

protocols.membrane.geometry.EmbeddingDef: Computing membrane embedding from TMspan 163 to 191
protocols.membrane.geometry.EmbeddingDef: [ WARNING ] If your starting PDB is already translated into the
protocols.membrane.geometry.EmbeddingDef: [ WARNING ] membrane, then your TMspan does not span the membrane!
protocols.membrane.geometry.EmbeddingDef: Computing membrane embedding from TMspan 204 to 229
protocols.membrane.geometry.Embedding: Span Embedding:
protocols.membrane.geometry.EmbeddingDef: Embedding: center: (-5.4075, -10.09, -6.554), normal: (-0.549385, -0.784197, 0.288464)
protocols.membrane.geometry.EmbeddingDef: Embedding: center: (-17.139, -19.121, -10.501), normal: (-0.622645, -0.656732, 0.42546)
protocols.membrane.geometry.EmbeddingDef: Embedding: center: (-22.7125, -14.013, -18.656), normal: (-0.750009, -0.549474, 0.368192)
protocols.membrane.geometry.EmbeddingDef: Embedding: center: (-25.6355, -3.841, -26.906), normal: (-0.68486, -0.568426, 0.455915)
protocols.membrane.ge

protocols.membrane.geometry.EmbeddingDef: Computing membrane embedding from TMspan 8 to 35
protocols.membrane.geometry.EmbeddingDef: Computing membrane embedding from TMspan 39 to 56
protocols.membrane.geometry.EmbeddingDef: Computing membrane embedding from TMspan 77 to 99
protocols.membrane.geometry.EmbeddingDef: [ WARNING ] If your starting PDB is already translated into the
protocols.membrane.geometry.EmbeddingDef: [ WARNING ] membrane, then your TMspan does not span the membrane!
protocols.membrane.geometry.EmbeddingDef: Computing membrane embedding from TMspan 104 to 125
protocols.membrane.geometry.EmbeddingDef: [ WARNING ] If your starting PDB is already translated into the
protocols.membrane.geometry.EmbeddingDef: [ WARNING ] membrane, then your TMspan does not span the membrane!
protocols.membrane.geometry.EmbeddingDef: Computing membrane embedding from TMspan 134 to 151
protocols.membrane.geometry.EmbeddingDef: [ WARNING ] If your starting PDB is already translated into the
p

protocols.membrane.SetMembranePositionMover:    Edge   	   Jump     Jump #
          	0236--0001  001
0001--0235
protocols.membrane.AddMembraneMover: Final foldtree: Is membrane fixed? 1
protocols.membrane.AddMembraneMover:    Edge   	   Jump     Jump #
          	0236--0001  001
0001--0235
protocols.membrane.MembranePositionMoverFromTopologyMover: Reordering foldtree:
protocols.membrane.MembranePositionMoverFromTopologyMover: Starting foldtree: Is membrane fixed? 0
protocols.membrane.MembranePositionMoverFromTopologyMover:    Edge   	   Jump     Jump #
          	0001--0236  001
0001--0235
protocols.membrane.MembranePositionMoverFromTopologyMover: Computing initial membrane position from structure...
protocols.membrane.geometry.Embedding: Constructing Embedding object from topology and pose
protocols.membrane.geometry.Embedding: Computing membrane embedding from TMspans:
protocols.membrane.geometry.EmbeddingDef: Computing membrane embedding from TMspan 8 to 35
protocols.membrane.geome

core.conformation.membrane.SpanningTopology: Span 4: start: 104, end: 125 orientation: 1
core.conformation.membrane.SpanningTopology: Span 5: start: 134, end: 151 orientation: 1
core.conformation.membrane.SpanningTopology: Span 6: start: 163, end: 191 orientation: 1
core.conformation.membrane.SpanningTopology: Span 7: start: 204, end: 229 orientation: 1
protocols.membrane.SetMembranePositionMover: Calling SetMembranePositionMover
protocols.membrane.SetMembranePositionMover: Starting foldtree: Is membrane fixed? 1
protocols.membrane.SetMembranePositionMover:    Edge   	   Jump     Jump #
          	0236--0001  001
0001--0235
protocols.membrane.SetMembranePositionMover: Final foldtree: Is membrane fixed? 1
protocols.membrane.SetMembranePositionMover:    Edge   	   Jump     Jump #
          	0236--0001  001
0001--0235
protocols.membrane.AddMembraneMover: Final foldtree: Is membrane fixed? 1
protocols.membrane.AddMembraneMover:    Edge   	   Jump     Jump #
          	0236--0001  001
0001-

core.chemical: [ WARNING ] Patch AcetylatedNtermProteinFull implies it can apply to residue type DRYS, but actually applying it fails.
core.chemical: [ WARNING ]    You may want to check your patch definitions.
core.chemical: [ WARNING ] Patch AcetylatedNtermProteinFull implies it can apply to residue type DRYS, but actually applying it fails.
core.chemical: [ WARNING ]    You may want to check your patch definitions.
protocols.membrane.AddMembraneMover: =====================================================================
protocols.membrane.AddMembraneMover: ||           WELCOME TO THE WORLD OF MEMBRANE PROTEINS...          ||
protocols.membrane.AddMembraneMover: =====================================================================
protocols.membrane.AddMembraneMover: No membrane residue was found
protocols.membrane.AddMembraneMover: Adding a new membrane residue to the pose
protocols.membrane.AddMembraneMover: Adding a membrane residue representing the position of the membrane after 

In [8]:
#score mutants 
sfxn(S184_mut), sfxn(T188_mut), sfxn(Y191_mut), sfxn(F195_mut), sfxn(Q183_mut)
#These scores are actually different!!!

core.pack.rotamers.SingleLigandRotamerLibraryCreator: [ WARNING ] using PDB_ROTAMERS for non-ligand ResidueType!
core.pack.rotamers.SingleLigandRotamerLibrary: Read in 54 rotamers from /usr/local/lib/python2.7/dist-packages/pyrosetta-2018.12+release.5ecebca-py2.7-linux-x86_64.egg/pyrosetta/database/chemical/residue_type_sets/fa_standard/residue_types/spin_labels/R1A_rotamers_nonclashing_moe_saved.pdb !


(-830.7329274519635,
 -831.481428708941,
 -803.6070559330046,
 -823.5141075092561,
 -813.4913451245204)

In [9]:
#Finding a ddg!!!!
ddg_1 = sfxn(S184_mut)-sfxn(pose)
ddg_2 = sfxn(T188_mut)-sfxn(pose)
ddg_3 = sfxn(Y191_mut)-sfxn(pose)
ddg_4 = sfxn(F195_mut)-sfxn(pose)
ddg_5 = sfxn(Q183_mut)-sfxn(pose)
ddg_1, ddg_2, ddg_3, ddg_4, ddg_5

(-39.92150321217207,
 -40.6700044691496,
 -12.795631693213181,
 -32.702683269464615,
 -22.679920884728972)

In [10]:
#Trying another scorefxn
sfxn2 = create_score_function( "mpframework_fa_2007.wts" )
sfxn2(pose)

-578.4273553355353

In [11]:
sfxn2(S184_mut), sfxn2(T188_mut), sfxn2(Y191_mut), sfxn2(F195_mut), sfxn2(Q183_mut)

(-615.8759362654454,
 -629.2227078845103,
 -599.5151866589366,
 -606.958490628491,
 -610.4295355535573)

In [12]:
ddg_1 = sfxn2(S184_mut)-sfxn2(pose)
ddg_2 = sfxn2(T188_mut)-sfxn2(pose)
ddg_3 = sfxn2(Y191_mut)-sfxn2(pose)
ddg_4 = sfxn2(F195_mut)-sfxn2(pose)
ddg_5 = sfxn2(Q183_mut)-sfxn2(pose)
ddg_1, ddg_2, ddg_3, ddg_4, ddg_5

(-37.44858092991012,
 -50.795352548975075,
 -21.08783132340136,
 -28.531135292955696,
 -32.00218021802209)

### Below, I have figured out a way to mutate a residue in proteorhodopsin to R1A.
### Now, I just need to figure out how to repack the residues within 8 angstroms of the mutation.
### After repacking, I will just need to calculate the mutant score.

In [ ]:
#Mutating pose (needs updating 180720)
WT_pose = pose_from_file("WT_TCM_pR.pdb")
mut_pose = pose_from_file("TCM_pR.pdb")
R1A_pose = pose_from_sequence('Z[R1A]', res_type="fa_standard")
R1A_res = R1A_pose.residue(1)
mut_pose.replace_residue(167, R1A_res, True)

### End Current Work 180720

In [ ]:
#Show WT vs Mutant
from pyrosetta.toolbox import mutate_residue
from pyrosetta import PyMOLMover
my_ip = os.environ["my_ip"]
#my_ip = '127.0.0.1'
pymover = PyMOLMover(my_ip, 65000)

pymover.apply(WT_pose)

pymover.apply(mut_pose)

In [ ]:
if repack_sidechain:
            # annoying but apparently has to be done this way?...
            repack_task = pyrosetta.rosetta.core.pack.task.TaskFactory.create_packer_task(mut_pose)
            repack_task.restrict_to_repacking()
            repack_res_list = pyrosetta.rosetta.utility.vector1_bool()
            for i in range(1,mut_pose.total_residue()+1):
                repack_res_list.append(i == residue)
            repack_task.restrict_to_residues(repack_res_list)
            sidechain_PackRotamersMover = pyrosetta.rosetta.protocols.simple_moves.PackRotamersMover(self.scorefn,repack_task)
            
            initial_mut_score = self.scorefn(mut_pose)
            if self.verbose > 1:
                print "Packer task:"
                print repack_task
                print "Repacking mutated sidechain %s %s -> %s..." % (mut_pose.pdb_info().pose2pdb(residue),ref_res_name,aa_name),
            sidechain_PackRotamersMover.apply(mut_pose)
            repacked_mut_score = self.scorefn(mut_pose)
            if self.verbose > 1:
                print "initial E = %f repacked E = %f" % (initial_mut_score,repacked_mut_score)
        #self.verbose=tmp_verbose

        return mut_pose

        
    def packer_task_repack_in_radius(self,pose,residue,radius):
        """
        Build a packer task that repacks all residues with CAlphas within distance radius
    
        Might want to remake to take centroid side-chains?  Looks like SwitchResidueTypeMover has trouble w/ nsAAs...
        """
        pack_residues = self.residue_CAs_in_radius(pose,residue,radius)
        return self.make_packer_task_with_residues(pose,pack_residues)

    def make_packer_task_with_residues(self,pose,residues=None):
        """
        Builds a packer task with the specified residues activated for repacking.
    
        Did this to avoid PackerTask.temporarily_* methods which apparently we're not supposed to use
        """
        packer_task = pyrosetta.rosetta.core.pack.task.TaskFactory.create_packer_task(pose)
        packer_task.restrict_to_repacking()
        if self.verbose > 1:
            print residues
        if residues != None:
            # Vector1 doesn't translate booleans correctly in 
            # TACC version of PyRosetta; need to build utility.vector1_bool() directly
            #pack_residues = Vector1([x in residues for x in range(1,pose.total_residue())])
            pack_residues = pyrosetta.rosetta.utility.vector1_bool()
            for i in range(1,pose.total_residue() + 1):
                if self.restrict_to_chain and not (pose.pdb_info().chain(i) in self.restrict_to_chain):
                    continue
                else:
                    #print (i,i in residues)
                    pack_residues.append(i in residues)
            packer_task.restrict_to_residues(pack_residues)
        
        return packer_task

    def residue_CAs_in_radius(self,pose,centerAA,radius):
        """
        Get a list of residues with C-alpha atoms within 'radius' distance from centerAA'a C-alpha
        """
        centerAA_CA = pose.residue(centerAA).xyz('CA')
        repack_residues = []
        if self.verbose > 2:
            print >> sys.stderr, "residue_CA_in_radius: centerAA=%d, restrict_to_chain=%s" % (centerAA,str(self.restrict_to_chain))
        for i in range(1,pose.total_residue()):
            # This is a little fudgy since restrict_to_chain could be either a list or a string depending on the calling child class
            # 
            # Note that this may break if chains are not single characters
            if self.restrict_to_chain and not (pose.pdb_info().chain(i) in self.restrict_to_chain):
                continue
            test_CA = pose.residue(i).xyz('CA')
            displacement = centerAA_CA - test_CA
            distance = displacement.norm()
            if distance <= radius:
                repack_residues.append(i)
    
        return repack_residues
    

In [ ]:
#Trying to adapt PyRosetta_TACC_MPI
def mutate_aa(pose,residue,aa_name,orient_bb=True,repack_sidechain=True, clone_pose=True):
    mut_pose = pose
    if clone_pose:
        mut_pose = pyrosetta.Pose()
        mut_pose.assign(pose)
    res = mut_pose.residue(residue)
    ref_res_name = res.name()
    mut_res_type = None
    rts = None
    chm = chemical.ChemicalManager.get_instance()
    try:
        rts = chm.residue_type_set("fa_standard").get()
    except AttributeError:
        rts = chm.residue_type_set("fa_standard")
    if residue == 1:
        try:
            mut_res_type = rts.get_residue_type_with_variant_added(mut_res_type,chemical.VariantType.LOWER_TERMINUS_VARIANT)
        except AttributeError:
                mut_res_type = rts.get_residue_type_with_variant_added(mut_res_type,"LOWER_TERMINUS")
    elif residue == mut_pose.total_residue():
        try:
            mut_res_type = rts.get_residue_type_with_variant_added(mut_res_type,chemical.VariantType.UPPER_TERMINUS_VARIANT)                
        except AttributeError:
            mut_res_type = rts.get_residue_type_with_variant_added(mut_res_type,"UPPER_TERMINUS")
    mut_res = conformation.ResidueFactory.create_residue( mut_res_type )
    mut_pose.replace_residue(residue,mut_res,orient_backbone=orient_bb)
    if repack_sidechain:
            # annoying but apparently has to be done this way?...
            repack_task = pyrosetta.rosetta.core.pack.task.TaskFactory.create_packer_task(mut_pose)
            repack_task.restrict_to_repacking()
            repack_res_list = pyrosetta.rosetta.utility.vector1_bool()
            for i in range(1,mut_pose.total_residue()+1):
                repack_res_list.append(i == residue)
            repack_task.restrict_to_residues(repack_res_list)
            sidechain_PackRotamersMover = pyrosetta.rosetta.protocols.simple_moves.PackRotamersMover(self.scorefn,repack_task)
            
            initial_mut_score = self.scorefn(mut_pose)
            if self.verbose > 1:
#                 print "Packer task"
                repack_task
                #print "Repacking mutated sidechain %s %s -> %s..." % 
                print (mut_pose.pdb_info().pose2pdb(residue),ref_res_name,aa_name),
            sidechain_PackRotamersMover.apply(mut_pose)
            repacked_mut_score = self.scorefn(mut_pose)
            if self.verbose > 1:
#                print "initial E = %f repacked E = %f" % 
                print (initial_mut_score,repacked_mut_score)
        #self.verbose=tmp_verbose

    return mut_pose
    
    


In [ ]:
pose = pose_from_file("TCM_pR.pdb")
residue = 184
aa_name = "R1A"

In [ ]:
mutate_aa(pose,residue,aa_name,orient_bb=True,repack_sidechain=True, clone_pose=True)